In [ ]:
sqlite3 qChron.db

In [1]:
import csv
rows = []
with open('surAyPosStrAdvWrdMD.tsv') as f:
    reader = csv.DictReader(f, delimiter='\t', )
    for row in reader:
        rows.append(row)

In [2]:
rows[0]

{'surah': '96',
 'ayah': '1',
 'position': '1',
 'wrd': 'ٱقْرَأْ',
 'mean': 'Read',
 'striDLs': '[{"stri": "qrA", "strTyp": "root", "poSp": "V", "frm": "i"}]'}

In [17]:
import json
for row in rows:
    surah = row["surah"]
    ayah = row["ayah"]
    position = row["position"]
    wrd = row["wrd"]
    striDLs = json.loads(row["striDLs"])
    lenStriDLs = len(striDLs)
    if lenStriDLs == 0:
        # print(f"at {surah}:{ayah}:{position}, {wrd} has no string")
        pass
    elif lenStriDLs < 0:
        print(f"weird negative strings at {surah}:{ayah}:{position}, {wrd}: \n{striDLs}")
    else:
        isStem = False
        for strD in striDLs:
            if strD["strTyp"] == "stem":
                isStem = True
        if isStem == True:
            # print(f"stem at {surah}:{ayah}:{position}, {wrd}: \n{striDLs}")
            pass
        else:
            if lenStriDLs == 1:
                pass
            elif lenStriDLs == 2:
                strObj1 = striDLs[0]
                strObj2 = striDLs[1]
                strObj1Len = len(strObj1)
                strObj2Len = len(strObj2)
                if strObj1Len != strObj2Len:
                    print(f" missing or extra info at {surah}:{ayah}:{position}, {wrd}: \n{striDLs}")
                else:
                    for k1 in ["strTyp"]:
                        if strObj1[k1] == strObj2[k1]:
                            print(f"weird double entry at {surah}:{ayah}:{position}, {wrd}\n{striDLs}")
                    for k2 in ["frm", "poSp"]:
                        if strObj1[k2] != strObj2[k2]:
                            print(f"weird conflicting entry at {surah}:{ayah}:{position}, {wrd}\n{striDLs}")                
            elif lenStriDLs > 2:
                print(f"{lenStriDLs} strings at {surah}:{ayah}:{position}, {wrd}: \n{striDLs}")
                pass

3 strings at 89:6:6, بِعَادٍ: 
[{'stri': 'Ewd', 'strTyp': 'root', 'poSp': 'PN', 'frm': 'i'}, {'stri': 'EaAd', 'strTyp': 'lem', 'poSp': 'PN', 'frm': 'i'}, {'stri': 'EaAd2', 'strTyp': 'lem', 'poSp': 'PN', 'frm': 'i'}]
3 strings at 53:41:4, ٱلْأَوْفَىٰ: 
[{'stri': 'wfy', 'strTyp': 'root', 'poSp': 'ADJ', 'frm': 'i'}, {'stri': '>awofaY`', 'strTyp': 'lem', 'poSp': 'ADJ', 'frm': 'i'}, {'stri': '>awofaY`2', 'strTyp': 'lem', 'poSp': 'ADJ', 'frm': 'i'}]
3 strings at 53:50:3, عَادًا: 
[{'stri': 'Ewd', 'strTyp': 'root', 'poSp': 'PN', 'frm': 'i'}, {'stri': 'EaAd', 'strTyp': 'lem', 'poSp': 'PN', 'frm': 'i'}, {'stri': 'EaAd2', 'strTyp': 'lem', 'poSp': 'PN', 'frm': 'i'}]
3 strings at 50:13:1, وَعَادٌ: 
[{'stri': 'Ewd', 'strTyp': 'root', 'poSp': 'PN', 'frm': 'i'}, {'stri': 'EaAd', 'strTyp': 'lem', 'poSp': 'PN', 'frm': 'i'}, {'stri': 'EaAd2', 'strTyp': 'lem', 'poSp': 'PN', 'frm': 'i'}]
3 strings at 54:18:2, عَادٌ: 
[{'stri': 'Ewd', 'strTyp': 'root', 'poSp': 'PN', 'frm': 'i'}, {'stri': 'EaAd', 'strTyp': 

In [ ]:
import sqlite3
conn = sqlite3.connect('qCrhon.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS suAyPosT (
        suAyPos TEXT PRIMARY KEY NOT NULL,
        surah INTEGER NOT NULL,
        ayah INTEGER NOT NULL,
        position INTEGER NOT NULL,
        wrd TEXT NOT NULL,
        mean TEXT NOT NULL DEFAULT '',
        striDLs TEXT NOT NULL DEFAULT '[]',
    )
''')

In [ ]:
cursor.execute('''
    CREATE TABLE striT (
        stri TEXT PRIMARY KEY,
    );
''')

In [ ]:
cursor.execute('''
    CREATE TABLE suAyPos_stri (
        suAyPos TEXT,
        stri TEXT,
        PRIMARY KEY (suAyPos, stri),
        FOREIGN KEY (suAyPos) REFERENCES suAyPosT(suAyPos),
        FOREIGN KEY (stri) REFERENCES striT(stri)
    );
''')

In [ ]:
# backend/database/__init__.py

from sqlalchemy import Column, Integer, String, ForeignKey, Table, create_engine
# from sqlalchemy.ext.declarative import declarative_base

from sqlalchemy.orm import declarative_base, relationship, sessionmaker

Base = declarative_base()

from . import *

In [ ]:
# Association table
suAyPos_stri = Table(
    'suAyPos_stri', Base.metadata,
    Column('suAyPos', ForeignKey('suAyPosT.suAyPos'), primary_key=True),
    Column('stri', ForeignKey('striT.stri'), primary_key=True)
)

In [ ]:
# backend/database/models/suAyPosT.py

# backend/database/models/suAyPosT.py
from backend.database.models import Column, Integer, String
#, create_engine
# from sqlalchemy.ext.declarative import declarative_base
from backend.database import Base

class SuAyPosT(Base):
    __tablename__ = 'suAyPosT'
    suAyPos = Column(String, primary_key=True)
    stris = relationship("StriT", secondary=suAyPos_stri, back_populates="suaypos_list")

class suAyPosTC(Base):
    __tablename__ = 'suAyPosT'

    suAyPos = Column(String, primary_key=True, nullable=False)
    surah = Column(Integer, nullable=False)
    ayah = Column(Integer, nullable=False)
    position = Column(Integer, nullable=False)
    wrd = Column(String, nullable=False)  
    mean = Column(String, nullable=False, default='')  
    striDLs = Column(String, nullable=False, default='[]')


In [2]:
suAyPosTC

__main__.suAyPosTC

In [ ]:
class StriT(Base):
    __tablename__ = 'striT'
    stri = Column(String, primary_key=True)
    suaypos_list = relationship("SuAyPosT", secondary=suAyPos_stri, back_populates="stris")
